In [1]:
from fastai.vision.all import *
import fastai
import timm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
import torch
torch.cuda.set_device(0)

In [3]:
df = pd.read_csv('data/wheat_new.csv',decimal=',')

In [4]:
name =  'swinv2_cr_base_384'

In [5]:
df

,Unnamed: 0,file,DAS,plot nº,photo nº,species,water_treatment,management,plot m2,avg_water (L m-2),...,P (kg microplot-1),K (kg microplot-1),avg_height,GA,GGA,CSI,NGRDIveg,TGIveg,final yield (kg),dataset
0,0,20220517_1 (1).JPG,96,1,(1).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.959407,0.904393,5.734098,0.1324,2569.2709,9.15,train
1,1,20220517_1 (2).JPG,96,1,(2).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.963030,0.905523,5.971505,0.1316,2648.0105,9.15,train
2,2,20220517_1 (3).JPG,96,1,(3).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.953708,0.898722,5.765430,0.1331,2736.4337,9.15,train
3,3,20220517_1 (4).JPG,96,1,(4).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.957117,0.898772,6.095965,0.1267,2550.8238,9.15,train
4,4,20220602_1 (1).JPG,112,1,(1).,wheat,ww,conv,12,3096.2375,...,0.072,0.072,46.9715,0.892348,0.791199,11.335180,0.1008,1880.2128,9.15,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,251,20220615_25 (4).JPG,125,25,(4).,wheat,d,conv,12,2752.8000,...,0.072,0.072,55.2110,0.788544,0.323428,58.984136,0.0491,2277.6824,7.95,test
252,252,20220701_25 (1).JPG,141,25,(1).,wheat,d,conv,12,2752.8000,...,0.072,0.072,55.2110,0.031868,0.009072,71.532905,0.0342,946.1760,7.95,test
253,253,20220701_25 (2).JPG,141,25,(2).,wheat,d,conv,12,2752.8000,...,0.072,0.072,55.2110,0.025828,0.006849,73.483356,0.0314,867.4491,7.95,test
254,254,20220701_25 (3).JPG,141,25,(3).,wheat,d,conv,12,2752.8000,...,0.072,0.072,55.2110,0.022065,0.004773,78.367629,0.0196,855.3556,7.95,test


In [6]:
scaler = MinMaxScaler()

In [7]:
df.columns

Index(['Unnamed: 0', 'file', 'DAS', 'plot nº', 'photo nº', 'species',
       'water_treatment', 'management', 'plot m2', 'avg_water (L m-2)',
       'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
       'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
       'final yield (kg)', 'dataset'],
      dtype='object')

In [8]:
dftrain = df[(df.dataset=='train')]
dftrain.loc[:,['plot m2', 'avg_water (L m-2)',
       'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
       'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
       'final yield (kg)']] = scaler.fit_transform(dftrain[['plot m2', 'avg_water (L m-2)',
       'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
       'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
       'final yield (kg)']])
dfvalid = df[(df.dataset=='validation')]
dftest = df[(df.dataset=='test')]
dfvalid.loc[:,['plot m2', 'avg_water (L m-2)',
       'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
       'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
       'final yield (kg)']] = scaler.transform(dfvalid[['plot m2', 'avg_water (L m-2)',
       'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
       'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
       'final yield (kg)']])        
dftest.loc[:,['plot m2', 'avg_water (L m-2)',
       'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
       'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
       'final yield (kg)']] = scaler.transform(dftest[['plot m2', 'avg_water (L m-2)',
       'N (kg microplot-1)', 'P (kg microplot-1)', 'K (kg microplot-1)',
       'avg_height', 'GA', 'GGA', 'CSI', 'NGRDIveg', 'TGIveg',
       'final yield (kg)']])                   

In [9]:
dftrain = pd.concat([dftrain,dfvalid])

In [10]:
df = pd.concat([dftrain,dftest])

In [11]:
dftrain.loc[:,'dataset'] =  dftrain['dataset'].apply(lambda x: x=='validation')
df.loc[:,'dataset'] =  df['dataset'].apply(lambda x: x=='test')

In [12]:
# dftrain.loc[:,'final yield (kg)'] = dftrain['final yield (kg)'].apply(lambda x : (x - 0.8579)/(10.55-0.8579))
# df.loc[:,'final yield (kg)'] = df['final yield (kg)'].apply(lambda x : (x - 0.8579)/(10.55-0.8579))

In [13]:
from fastai.vision.all import *
callbacks = [
    ShowGraphCallback(),
    # EarlyStoppingCallback(patience=5),
    SaveModelCallback(fname=name+'_all_minmax_new'), #convnext_yield_all_minmax
    # ReduceLROnPlateau()
]

In [14]:
from fastai.metrics import mae,rmse,mse

In [15]:
db = DataBlock(blocks = (ImageBlock,CategoryBlock(),CategoryBlock(),CategoryBlock(),
                         RegressionBlock(),RegressionBlock(),RegressionBlock(),RegressionBlock(),
                         RegressionBlock(),RegressionBlock(),RegressionBlock(),RegressionBlock(),
                         RegressionBlock(),RegressionBlock(),RegressionBlock(),RegressionBlock()),
                 get_x=[ColReader('file',pref='wheat_all/'),
                        ColReader('species'),
                        ColReader('water_treatment'),
                        ColReader('management'),
                        ColReader('plot m2'),
                        ColReader('avg_water (L m-2)'),
                        ColReader('N (kg microplot-1)'),
                        ColReader('P (kg microplot-1)'),
                        ColReader('K (kg microplot-1)'),
                        ColReader('avg_height'),
                        ColReader('GA'),
                        ColReader('GGA'),
                        ColReader('CSI'),
                        ColReader('NGRDIveg'),
                        ColReader('TGIveg')
                       ],
                 get_y=[ColReader('final yield (kg)')],
                 splitter=ColSplitter(col='dataset'),    
                 item_tfms = Resize(512),
                 batch_tfms=[*aug_transforms(size=384,flip_vert=True,
                                            max_zoom=1.0), Normalize.from_stats(*imagenet_stats)],
               n_inp=15)
dls = db.dataloaders(dftrain,bs=2)

In [16]:
class MultiInputModel(Module):
  "A three-headed model given a `body` and `n` output features"
  def __init__(self, body:nn.Sequential):
    nf = num_features_model(nn.Sequential(*body.children()))
    self.body = body
    self.image = create_head(nf, 10)
    self.combine = nn.Sequential(
            nn.Linear(in_features=24, out_features=1, bias=False)
        )
    
    
  
  def forward(self, x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14):
    y = self.body(x0)
    y = self.image(y)
    final_yield = self.combine(torch.cat([y,torch.reshape(x1,(x1.shape[0],1)),
                                     torch.reshape(x2,(x2.shape[0],1)),
                                     torch.reshape(x3,(x3.shape[0],1)),
                                     torch.reshape(x4,(x4.shape[0],1)),
                                     torch.reshape(x5,(x5.shape[0],1)),
                                     torch.reshape(x6,(x6.shape[0],1)),
                                     torch.reshape(x7,(x7.shape[0],1)),
                                     torch.reshape(x8,(x8.shape[0],1)),
                                     torch.reshape(x9,(x9.shape[0],1)),
                                     torch.reshape(x10,(x10.shape[0],1)),
                                     torch.reshape(x11,(x11.shape[0],1)),
                                     torch.reshape(x12,(x12.shape[0],1)),
                                     torch.reshape(x13,(x13.shape[0],1)),
                                     torch.reshape(x14,(x14.shape[0],1))],1))
    return final_yield

In [17]:
from wwf.vision.timm import *

body = create_timm_body(name, pretrained=True)
# body = create_body(resnet50, pretrained=True)
net = MultiInputModel(body)

/opt/anaconda/envs/fastai/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
No pretrained weights exist or were found for this model. Using random initialization.


StopIteration: 

In [ ]:
learn = Learner(dls, net, loss_func=MSELossFlat(), metrics=[mae,rmse,mse],cbs=callbacks)

In [ ]:
# learn.lr_find()

In [ ]:
learn.freeze()

In [ ]:
learn.fit_one_cycle(30,lr_max=1e-3)

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(30,lr_max=1e-4)

In [ ]:
dls = db.dataloaders(df,bs=4)
learn.dls=dls

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
preds,gt = learn.get_preds()

In [ ]:
mean_absolute_error(gt*(10.55-0.8579)+0.8579,preds*(10.55-0.8579)+0.8579)

In [ ]:
mean_squared_error(gt*(10.55-0.8579)+0.8579,preds*(10.55-0.8579)+0.8579)

In [ ]:
math.sqrt(mean_squared_error(gt*(10.55-0.8579)+0.8579,preds*(10.55-0.8579)+0.8579))